<a href="https://colab.research.google.com/github/Herceg00/sphere_nn/blob/master/Task10/Practice%20task%208%2C%20Advanced%20NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Семинар 8: "Современные модели для NLP"

ФИО: Личманов Дмитрий Игоревич

### На семинаре мы разберем [код трансфомера на pytorch](https://nlp.seas.harvard.edu/2018/04/03/attention.html)

###  ДЗ [3 балла]

Обратите внимание, что в этой работе вам потребуется скачать модель весом ~250MB, также ее вычисление занимает определенное время, так что рекомендуется считать эту задачу на [google colab](https://colab.research.google.com/).

In [67]:
import torch
from tqdm import tqdm_notebook as tqdm
!pip install transformers
from transformers import *

In [68]:
MODEL = (DistilBertForMaskedLM, DistilBertTokenizer, 'distilbert-base-cased')

model_class, tokenizer_class, pretrained_weights = MODEL
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [69]:
input_ids = tokenizer.encode("Here is some text to encode", add_special_tokens=True)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
print(input_ids)

[101, 3446, 1110, 1199, 3087, 1106, 4035, 13775, 102]


In [70]:
tokenizer.decode(input_ids)

'[CLS] Here is some text to encode [SEP]'

In [71]:
input_ids[4] = tokenizer.mask_token_id
tokenizer.decode(input_ids)

'[CLS] Here is some [MASK] to encode [SEP]'

In [72]:
input_ids

[101, 3446, 1110, 1199, 103, 1106, 4035, 13775, 102]

In [73]:
input_batch = torch.tensor(input_ids).unsqueeze(0) # batch_size 1
with torch.no_grad():
    res = model(input_batch)[0]

In [74]:
prob = torch.nn.functional.softmax(res, dim=-1)
new_ids = prob.max(-1)[1]

In [75]:
tokenizer.decode(new_ids.numpy()[0, :].tolist())

'. here is some way to encode.'

In [78]:
GPT_TEXTS = [
    "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.",
    "A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown."
    ]

Ваша задача - сгенерировать продолжение текстов, на которых демонстрировалась работа GPT-2 с помощью загруженной модели (DistillBERT). Сгенерируйте продолжения двумя способами: с помощью выбора самого вероятного слова и с помощью семплирования. Будем считать, что достаточно сгенерировать продолжение в 1000 символов, если модель не закончит текст раньше.

Max prob

In [108]:
initial_text = GPT_TEXTS[0]
encoded_text = tokenizer.encode(GPT_TEXTS[0], add_special_tokens=True)
encoded_text.pop()

for i in tqdm(range(100)):
    encoded_text.append(tokenizer.mask_token_id)
    input_sequence = torch.tensor(encoded_text).unsqueeze(0)
    prediction = model(input_sequence)[0]
    probabilities = torch.nn.functional.softmax(prediction, dim = -1)
    predicted_index = probabilities.max(-1)[1][0]
    new_token = predicted_index[-1].item()
    encoded_text[-1] = new_token
    if (new_token == 102): #Токен конца
        break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [109]:
tokenizer.decode(encoded_text) # Можно было и остановиться

'[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. unfortunately, scientists discovered dinosaurs like dinosaurs like dinosaurs like dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs dinosaurs 

In [117]:
initial_text = GPT_TEXTS[1]
encoded_text = tokenizer.encode(GPT_TEXTS[1], add_special_tokens=True)
encoded_text.pop()

for i in tqdm(range(100)):
    encoded_text.append(tokenizer.mask_token_id)
    input_sequence = torch.tensor(encoded_text).unsqueeze(0)
    prediction = model(input_sequence)[0]
    probabilities = torch.nn.functional.softmax(prediction, dim = -1)
    predicted_index = probabilities.max(-1)[1][0]
    new_token = predicted_index[-1].item()
    encoded_text[-1] = new_token
    if (new_token == 102): #Токен конца
        break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [118]:
tokenizer.decode(encoded_text) # Можно было и остановиться

'[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown. whereabouts unknown or unknown or unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown unknown'

Sampling


In [110]:
from torch.distributions.categorical import Categorical

In [111]:
initial_text = GPT_TEXTS[0]
encoded_text = tokenizer.encode(GPT_TEXTS[0], add_special_tokens=True)
encoded_text.pop()

for i in tqdm(range(100)):
    encoded_text.append(tokenizer.mask_token_id)
    input_sequence = torch.tensor(encoded_text).unsqueeze(0)
    prediction = model(input_sequence)[0]
    probabilities = torch.nn.functional.softmax(prediction, dim = -1)
    m = Categorical(probabilities)
    predicted_index = m.sample()[0] 
    new_token = predicted_index[-1].item()
    encoded_text[-1] = new_token
    if (new_token == 102): #Токен конца
        break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [112]:
tokenizer.decode(encoded_text) 

'[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. equally stunning dinosaurs displayed magnificent wings and hornsaurus dinosaurs appeared. dinosaur dinosaurs dinosaur dinosaurs dinosaurs dinosaurs and enthusiastic scientists attract star spectacular fireworks famous in the museum fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks fireworks 

In [115]:
initial_text = GPT_TEXTS[1]
encoded_text = tokenizer.encode(GPT_TEXTS[1], add_special_tokens=True)
encoded_text.pop()

for i in tqdm(range(100)):
    encoded_text.append(tokenizer.mask_token_id)
    input_sequence = torch.tensor(encoded_text).unsqueeze(0)
    prediction = model(input_sequence)[0]
    probabilities = torch.nn.functional.softmax(prediction, dim = -1)
    m = Categorical(probabilities)
    predicted_index = m.sample()[0] 
    new_token = predicted_index[-1].item()
    encoded_text[-1] = new_token
    if (new_token == 102): #Токен конца
        break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [116]:
tokenizer.decode(encoded_text) 

'[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown. vehicle lost sector 2 first carriage pinned strengthening procedures certification process manual certification test trail bike safety safety explosive gloves safety explosives requiring instantly safety protective parts for steel mills safety products and ready consumed safety safer fire resistant ensure safety safety atmosphere safety burns marijuana safety Extreme manufacture safety safety deadly chemicals poison poisonous poisonous chemical animals poisonous poisonous poison smelling poisonous chemicals lethal poison breath poisonous ammunition safety poisoning safety flesh lethal hunger weapon poison kill poison poison poison poison hunting poison poison poison poison poison poison poison poison poison poison poison poison poison poison poison poison poison'

#### Feedback (опционально)

Здесь вы можете оставить список опечаток из лекции или семинара:

Здесь вы можете оставить комментарии по лекции или семинару: